<a href="https://colab.research.google.com/github/tipiorgup/Tutorial_IDP_SAPs/blob/main/Simulation%20code/Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import shutil
try:
    if os.path.exists('Tutorial_IDP_SAPs'):
        print("Found existing directory, attempting to remove...")
        shutil.rmtree('Tutorial_IDP_SAPs', ignore_errors=True)
        print("Directory removed successfully")
    else:
        print("Directory doesn't exist")
except Exception as e:
    print(f"Error while trying to remove directory: {e}")

Found existing directory, attempting to remove...
Directory removed successfully


In [2]:
from google.colab import output
output.enable_custom_widget_manager()

In [20]:
!git clone https://github.com/tipiorgup/Tutorial_IDP_SAPs.git
!pip install openmm
!pip install MDAnalysis
!pip install mdtraj
!pip install nglview
!pip install ipywidgets==7.7.1 jupyter_client==6.1.12 notebook==6.5.5
import ipywidgets as widgets
from IPython.display import display
import nglview
import mdtraj

Cloning into 'Tutorial_IDP_SAPs'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 131 (delta 44), reused 93 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (131/131), 87.59 KiB | 2.25 MiB/s, done.
Resolving deltas: 100% (44/44), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.6 MB/s eta 0:00:00
  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached notebook-7.3.2-py3-none-any.whl.metadata (10 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyter_server-2.15.0-py3-none-any.whl.metadata (8.4 kB)
  Using cached jupyter_client-8.6.3-py3-none-any.whl.metadata (8.3 kB)
Using cached ipywidgets-8.1.5-py3-none-any.whl (139 kB)
Using cach

  Using cached ipywidgets-7.7.1-py2.py3-none-any.whl.metadata (1.9 kB)
  Using cached jupyter_client-6.1.12-py3-none-any.whl.metadata (4.3 kB)
  Using cached notebook-6.5.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached widgetsnbextension-3.6.10-py2.py3-none-any.whl.metadata (1.3 kB)
INFO: pip is looking at multiple versions of jupyter-server to determine which version is compatible with other requirements. This could take a while.
  Using cached jupyter_server-2.14.2-py3-none-any.whl.metadata (8.4 kB)
  Using cached jupyter_server-2.14.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached jupyter_server-2.14.0-py3-none-any.whl.metadata (8.4 kB)
  Using cached jupyter_server-2.13.0-py3-none-any.whl.metadata (8.4 kB)
  Using cached jupyter_server-2.12.5-py3-none-any.whl.metadata (8.4 kB)
  Using cached jupyter_server-2.12.4-py3-none-any.whl.metadata (8.4 kB)
  Using cached jupyter_server-2.12.3-py3-none-any.whl.metadata (8.4 kB)
INFO: pip is still looking at multiple versions of jupyte

In [4]:
os.chdir('Tutorial_IDP_SAPs/Simulation code')

In [5]:
global stored_values
stored_values = {
    'Simulation type': None,
    'Sequence': None,
    'Sequence 1': None,
    'Sequence 2': None,
    'Ratio': None,
    'Processed_Ratio': None,
    'peptide_pairs': None
}

# Create the checkbox widgets
checkbox_a = widgets.Checkbox(
    value=False,
    description='Single peptide',
    disabled=False
)

checkbox_b = widgets.Checkbox(
    value=False,
    description='Co-assembly',
    disabled=False
)

# Create text input for option A
text_input_a = widgets.Text(
    description='Sequence:',
    disabled=False,
    layout={'visibility': 'hidden', 'width': '800px'},
    placeholder='Enter sequence (up to 100 characters)'
)

# Create text inputs for option B
text_input_b1 = widgets.Text(
    description='Sequence 1:',
    disabled=False,
    layout={'visibility': 'hidden', 'width': '800px'},
    placeholder='Enter sequence 1 (up to 100 characters)'
)

text_input_b2 = widgets.Text(
    description='Sequence 2:',
    disabled=False,
    layout={'visibility': 'hidden', 'width': '800px'},
    placeholder='Enter sequence 2 (up to 100 characters)'
)

text_input_b3 = widgets.Text(
    description='Ratio:',
    disabled=False,
    layout={'visibility': 'hidden', 'width': '800px'},
    placeholder='Enter ratio'
)

def process_ratio(ratio_str):
    try:
        numbers = ratio_str.replace(' ', '').split('-')
        if len(numbers) == 2:
            num1 = int(numbers[0]) * 100
            num2 = int(numbers[1]) * 100
            return f"{num1}-{num2}"
    except:
        return ratio_str
    return ratio_str

def update_stored_values():
    if checkbox_a.value:
        stored_values['Simulation type'] = 'Single peptide'
        stored_values['Sequence'] = text_input_a.value
        stored_values['Sequence 1'] = text_input_a.value
        stored_values['Sequence 2'] = 'A'  # Constant A as Sequence 2
        stored_values['Ratio'] = '10-0'  # Default ratio
        stored_values['Processed_Ratio'] = process_ratio('10-0')  # Process default ratio
        if text_input_a.value:  # Only create if sequence exists
            stored_values['peptide_pairs'] = f"{text_input_a.value}-A"
    elif checkbox_b.value:
        stored_values['Simulation type'] = 'Co-assembly'
        stored_values['Sequence'] = None
        stored_values['Sequence 1'] = text_input_b1.value
        stored_values['Sequence 2'] = text_input_b2.value
        stored_values['Ratio'] = text_input_b3.value
        stored_values['Processed_Ratio'] = process_ratio(text_input_b3.value)
        if text_input_b1.value and text_input_b2.value:
            stored_values['peptide_pairs'] = f"{text_input_b1.value}-{text_input_b2.value}"

def on_checkbox_change(change):
    if change['owner'] == checkbox_a and change['new']:
        checkbox_b.value = False
        text_input_a.layout.visibility = 'visible'
        text_input_b1.layout.visibility = 'hidden'
        text_input_b2.layout.visibility = 'hidden'
        text_input_b3.layout.visibility = 'hidden'
    elif change['owner'] == checkbox_b and change['new']:
        checkbox_a.value = False
        text_input_a.layout.visibility = 'hidden'
        text_input_b1.layout.visibility = 'visible'
        text_input_b2.layout.visibility = 'visible'
        text_input_b3.layout.visibility = 'visible'

    update_stored_values()

def on_text_change(change):
    update_stored_values()

# Register callbacks
checkbox_a.observe(on_checkbox_change, names='value')
checkbox_b.observe(on_checkbox_change, names='value')
text_input_a.observe(on_text_change, names='value')
text_input_b1.observe(on_text_change, names='value')
text_input_b2.observe(on_text_change, names='value')
text_input_b3.observe(on_text_change, names='value')



In [6]:
widget = widgets.VBox([
    checkbox_a,
    checkbox_b,
    text_input_a,
    text_input_b1,
    text_input_b2,
    text_input_b3
])

In [13]:
display(widget)

In [14]:
stored_values

{'Simulation type': 'Single peptide',
 'Sequence': 'HHHWWW',
 'Sequence 1': 'HHHWWW',
 'Sequence 2': 'A',
 'Ratio': '10-0',
 'Processed_Ratio': '1000-0',
 'peptide_pairs': 'HHHWWW-A'}

In [15]:
#Check in src the file parsers.py, it will show all options available
ionic_strength='0.3'

In [16]:
# First check file
residues_file = 'src/residues.csv'

if not os.path.exists(residues_file):
    print(f"❌ Error: File '{residues_file}' not found!")
else:
    !python clean_pair_code.py --ratio {stored_values['Ratio']} \
        --ionic-strength {ionic_strength} \
        --ph 7.4 \
        --peptide-pair {stored_values['peptide_pairs']} \
        --run-all \
        --residues-file 'src/residues.csv' \
        --steps 1000

Cannot import pdbfixer. This may affect 3SPN2 model.
Processing pair: HHHWWW-A
Check contact with FastNS method. 
Successfully inserted 1 molecules.
Check contact with FastNS method. 
Successfully inserted 0 molecules.
/content/Tutorial_IDP_SAPs/Simulation code/src/simulation.py:98: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.038286503882254706' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  r.loc['H', 'q'] = 1. / (1 + 10**(args.ph - 6))


In [38]:
name = stored_values['peptide_pairs']
import numpy as np

traj = mdtraj.load_dcd(f'{name}/310/{name}_slab.dcd', top=f'{name}/start.pdb')
traj.xyz -= np.mean(traj.xyz, axis=1, keepdims=True)  # realign to the origin
view = nglview.show_mdtraj(traj)